In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [8]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [9]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [10]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [11]:
len(df.columns.tolist())

86

In [12]:
df.iloc[0:2,80:87]

,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,65687.0,38199.0,2000.0,61987.0,N,N
1,271427.0,60641.0,41200.0,49197.0,N,N


In [13]:
# columns to fix
# home_ownership, verification_status, issue_d, pymnt_plan, 
# initial_list_status, next_pymnt_d, application_type, hardship_flag
# debt_settlement_flag

In [14]:
df["home_ownership"].value_counts()

MORTGAGE    36219
RENT        24613
OWN          7346
ANY           639
Name: home_ownership, dtype: int64

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["home_ownership"] = le.fit_transform(df["home_ownership"])


In [16]:
df["verification_status"].value_counts()
df["verification_status"] = le.fit_transform(df["verification_status"])
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,1,105000.0,2,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,1,56000.0,2,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,3,92000.0,2,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,1,52000.0,0,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [17]:
df["issue_d"].value_counts()

Jan-2019    31041
Feb-2019    25579
Mar-2019    12197
Name: issue_d, dtype: int64

In [18]:
df["issue_d"] = le.fit_transform(df["issue_d"])
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,2,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,1,105000.0,2,2,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,1,56000.0,2,2,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,3,92000.0,2,2,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,1,52000.0,0,2,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [19]:
df["pymnt_plan"].value_counts()
df = df.drop("pymnt_plan",axis=1)

In [20]:
# Reference:
# initial_list_status, next_pymnt_d, application_type, hardship_flag
# debt_settlement_flag

In [21]:
df=pd.get_dummies(df, columns=["initial_list_status"],drop_first=True)

In [22]:
df["next_pymnt_d"].value_counts()

May-2019    42449
Apr-2019    26368
Name: next_pymnt_d, dtype: int64

In [23]:
df["next_pymnt_d"] = le.fit_transform(df["next_pymnt_d"])

In [24]:
df["application_type"].value_counts()

Individual    59206
Joint App      9611
Name: application_type, dtype: int64

In [25]:
df["application_type"] = le.fit_transform(df["application_type"])

In [26]:
df["hardship_flag"].value_counts()

N    68817
Name: hardship_flag, dtype: int64

In [27]:
df.drop("hardship_flag",axis=1, inplace=True)

In [28]:
df["debt_settlement_flag"].value_counts()

N    68817
Name: debt_settlement_flag, dtype: int64

In [29]:
df.drop("debt_settlement_flag",axis=1, inplace=True)

# Split the Data into Training and Testing

In [30]:
# Create our features
X = df.drop("loan_status", axis=1)


# Create our target
y = df["loan_status"]

In [31]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,dti,delinq_2yrs,inq_last_6mths,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,initial_list_status_w
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.000000
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.805542,21.778153,0.217766,0.497697,...,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.876121
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.714932,20.199244,0.718367,0.758122,...,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.329446
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.000000
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.000000,13.890000,0.000000,0.000000,...,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,1.000000
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,19.760000,0.000000,0.000000,...,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,1.000000
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,1.000000,26.660000,0.000000,1.000000,...,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,1.000000
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,999.000000,18.000000,5.000000,...,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,1.000000


In [32]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [36]:
# Resample the training data with the BalancedRandomForestClassifier


In [37]:
# Calculated the balanced accuracy score


0.6808843183002242


In [38]:
# Display the confusion matrix


[[   54    77]
 [ 1139 21440]]


In [39]:
# Print the imbalanced classification report


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.05      0.41      0.95      0.08      0.63      0.37       131
   low_risk       1.00      0.95      0.41      0.97      0.63      0.41     22579

avg / total       0.99      0.95      0.42      0.97      0.63      0.41     22710



In [40]:
# List the features sorted in descending order by feature importance


[(0.15140885211786187, 'total_rec_int'),
 (0.13854939232755675, 'last_pymnt_amnt'),
 (0.13564719188942637, 'total_pymnt'),
 (0.12873891393613698, 'total_rec_prncp'),
 (0.08554471604939386, 'total_pymnt_inv'),
 (0.03770558834059504, 'int_rate'),
 (0.018125397726536218, 'total_rev_hi_lim'),
 (0.01755089663764109, 'max_bal_bc'),
 (0.016809533664796206, 'issue_d'),
 (0.01185807321291461, 'out_prncp_inv'),
 (0.011652304240655978, 'out_prncp'),
 (0.010042541078217795, 'total_bal_il'),
 (0.009902993393409096, 'total_bc_limit'),
 (0.00954775083977164, 'installment'),
 (0.009441305417022563, 'mths_since_recent_inq'),
 (0.009404327817345756, 'dti'),
 (0.009161742238796509, 'tot_hi_cred_lim'),
 (0.008525508230648352, 'mort_acc'),
 (0.007752926897342076, 'revol_bal'),
 (0.007672742950083064, 'mths_since_rcnt_il'),
 (0.007643397914799658, 'inq_last_6mths'),
 (0.007355593085439861, 'open_acc_6m'),
 (0.0071607300822048685, 'tot_cur_bal'),
 (0.007126606559423642, 'il_util'),
 (0.006908442882191329, 'a

### Easy Ensemble AdaBoost Classifier

In [41]:
# Train the EasyEnsembleClassifier


In [42]:
# Calculated the balanced accuracy score


0.8826463081786798


In [43]:
# Display the confusion matrix


[[  113    18]
 [ 2197 20382]]


In [44]:
# Print the imbalanced classification report


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.05      0.86      0.90      0.09      0.88      0.78       131
   low_risk       1.00      0.90      0.86      0.95      0.88      0.78     22579

avg / total       0.99      0.90      0.86      0.94      0.88      0.78     22710

